# Local APIs for use with OpenRefine
The APIs are Jupyter cells with one API/URL per cell and an accompanying cell for response formatting. For general use of Jupyter Kernelgateway see https://jupyter-kernel-gateway.readthedocs.io/en/latest/getting-started.html.<br>
The APIs of one notebook are available until restart or the process is stopped with `kill`.  <br>
Replace the `KernelGatewayApp.seed_uri` with your notebook path. The port can be changed if necessary.<br>
Tested with Jupyterlab 3.6.3, Python 3.9.13.

**libraries used:**
- jupyter_kernel_gateway
- geopy (to demonstrate that other webservices can be used within such an API notebook – here OpenStreetMap Nominatim)

**starting kernelgateway from Terminal**
- `jupyter kernelgateway --KernelGatewayApp.api=kernel_gateway.notebook_http --KernelGatewayApp.seed_uri=/Users/admin/Documents/Jupyter/localAPI.ipynb --port=10100`

**killing kernelgateway from Terminal**
- `lsof -i :10100` to find the process PID (look for "pythonX.X XXXXX") 
- `kill [PID]`

## dec2gms service
run from OpenRefine with something like <br>`"http://127.0.0.1:10100/dec2gms?lat=" + cells['lat'].value + "&lon=" + value`<br> in column lon if there is a column lon and a column lat or adust column names. Output is encoded in UTF-8.
Test with http://127.0.0.1:10100/dec2gms?lat=50.928780&lon=11.589900

In [ ]:
# GET /dec2gms

import json

req = json.loads(REQUEST)

try:

    latitude = float(req['args']['lat'][0])
    longitude = float(req['args']['lon'][0])  
except:
    print("No coordinates provided.")

#latitude = 50.928780
#longitude = 11.589900

def decimallon_to_dms(decimal):
    degrees = abs(int(decimal))
    minutes_decimal = (abs(decimal) - degrees) * 60
    minutes = int(minutes_decimal)
    seconds = (minutes_decimal - minutes) * 60

    direction = "E" if decimal >= 0 else "W"
    
    dms = f"{direction} {degrees:03d}°{minutes:02d}'{seconds:.2f}''"
    return dms
    
def decimallat_to_dms(decimal):
    degrees = abs(int(decimal))
    minutes_decimal = (abs(decimal) - degrees) * 60
    minutes = int(minutes_decimal)
    seconds = (minutes_decimal - minutes) * 60

    direction = "N" if decimal >= 0 else "S"

    dms = f"{direction} {degrees:03d}°{minutes:02d}'{seconds:.2f}''"
    return dms

# Example usage

latitude_dms = decimallat_to_dms(latitude)
longitude_dms = decimallon_to_dms(longitude)

formatted_coordinates = f"{longitude_dms} / {latitude_dms}"

result = formatted_coordinates

print(result)

In [ ]:
# ResponseInfo GET /dec2gms
print(json.dumps({
"headers" : {
    "Content-Type": "text/plain; charset=utf-8"
    },
    "status" : 201
}))

## gms2dec service
run from OpenRefine with <br>`"http://127.0.0.1:10100/gms2dec?lat=" + cells['lat'].value + "&lon=" + value`<br> in column lon if there is a column lon and a column lat, else adjust column names. Output is encoded in UTF-8.
Test with http://127.0.0.1:10100/gms2dec?lat=50°56'5.824\"N&lon=10°59'24.112\"E

In [ ]:
# GET /gms2dec

from geopy.geocoders import Nominatim
import json

req = json.loads(REQUEST)

try:
    gms_latitude = req['args']['lat'][0]
    gms_longitude = req['args']['lon'][0]

except:
    print("No coordinates provided.")  

def convert_gms_to_decimal(gms_latitude, gms_longitude):
    geolocator = Nominatim(user_agent="name@server.com") #change user agent to your email address
    location = geolocator.geocode(f"{gms_latitude}, {gms_longitude}") 
    return location.latitude, location.longitude  

gms_latitude = gms_latitude.replace("\\", "")
gms_longitude = gms_longitude.replace("\\", "")


decimal_latitude, decimal_longitude = convert_gms_to_decimal(gms_latitude, gms_longitude)

print(f"{decimal_latitude} / {decimal_longitude}")


In [ ]:
# ResponseInfo GET /gms2dec
print(json.dumps({
"headers" : {
    "Content-Type": "text/plain; charset=utf-8"
    },
    "status" : 201
}))